# 协程

In [1]:
from inspect import getgeneratorstate

In [1]:
def simple_coroutine():
    print ('-> coroutine started')
    x = yield
    print ('-> coroutine received:', x)

my_coro = simple_coroutine()

In [2]:
next(my_coro)
my_coro.send(42)

-> coroutine started
-> coroutine received: 42


StopIteration: 

In [ ]:
# 协程可以身处四个状态中的一个，可以用inspect.getgeneratorstate(...)函数确定，该函数会返回下述字符串中的一个
# 'GEN_CREATED' 等待开始执行
# 'GEN_RUNNING' 解释器正在执行
# 'GEN_SUSPENDED' 在yield表达式暂停
# 'GEN_CLOSED' 执行结束

In [7]:
def simple_coro2(a):
    print ('-> Started: a =', a)
    b = yield a
    print ('-> Received b =', b)
    c = yield a + b
    print ('-> Received c =', c)

my_coro2 = simple_coro2(14)

print (getgeneratorstate(my_coro2))
print (next(my_coro2))

GEN_CREATED
-> Started: a = 14
14


In [8]:
print (getgeneratorstate(my_coro2))
print (my_coro2.send(28))

GEN_SUSPENDED
-> Received b = 28
42


In [9]:
print (getgeneratorstate(my_coro2))
print (my_coro2.send(99))

GEN_SUSPENDED
-> Received c = 99


StopIteration: 

In [10]:
print (getgeneratorstate(my_coro2))

GEN_CLOSED


In [2]:
# 使用协程计算移动平均值
# 这个无限循环表明，只要调用方不断把这个值发给这个协程，它就一直接收值
# 仅当调用方在协程上调用.close()方法，或者没有对协程的引用而被垃圾回收程序回收时，这个协程才会终止
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total / count

In [ ]:
# 使用协程必须用next(xxx)预激，如果嫌麻烦可以用装饰器@coroutine

In [4]:
# 终止协程和异常处理
coro_avg = averager()
next(coro_avg)
print (coro_avg.send(40))
print (coro_avg.send(50))
# 发送的不是数字，导致协程内部有异常抛出
print (coro_avg.send('spam'))

40.0
45.0


TypeError: unsupported operand type(s) for +=: 'float' and 'str'

In [ ]:
# 显式地把异常抛给协程，这两个方法是throw和close
generator.throw(exc_type[, exc_value[, traceback]])
# 致使生成器在暂停的yield表达式处抛出指定的异常
generator.close()
# 致使生成器在暂停的yield表达式处抛出GeneratorExit异常

In [2]:
class DemoException(Exception):
    pass

def demo_exc_handling():
    print ('-> coroutine started')
    while True:
        try:
            x = yield
        except DemoException:
            print ('*** DemoException handled. Continuing')
        else:
            print ('-> coroutine received: {!r}'.format(x))
    # 这行代码不会执行，因为只有未处理的异常才会终止那个无限循环
    # 一旦出现未处理的异常，协程会立即终止
    raise RuntimeError('This line should never run.')

In [5]:
exc_coro = demo_exc_handling()

In [6]:
print (next(exc_coro))

-> coroutine started
None


In [7]:
exc_coro.send(11)
exc_coro.send(22)

-> coroutine received: 11
-> coroutine received: 22


In [8]:
exc_coro.throw(DemoException)

*** DemoException handled. Continuing


In [ ]:
exc_coro.close() # 会终止协程
exc_coro.throw(ZeroDivisionError) # 会终止协程，并抛出异常

In [9]:
# 让协程返回值
from collections import namedtuple

Result = namedtuple('Result', 'count average')

def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total / count
    return Result(count, average)

In [12]:
coro_avg = averager()
next(coro_avg)

In [11]:
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)
coro_avg.send(None)

StopIteration: Result(count=3, average=15.5)

In [13]:
# 补获StopIteration异常，获取averager返回的值
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)
try:
    coro_avg.send(None)
except StopIteration as exc:
    result = exc.value

In [14]:
print (result)

Result(count=3, average=15.5)


In [ ]:
# 使用 yield from
def gen():
    for c in 'AB':
        yield c
    for i in range(1, 3):
        yield i
# 可以改写为
def gen():
    yield from 'AB'
    yield from range(1,3)

In [ ]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')

# 子生成器
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total / count
    return Result(count, average)

# 委派生成器
def grouper(results, key):
    while True:
        results[key] = yield from averager()

# 客户端代码，即调用方
def main(data):
    results = {}
    for key, values in data.items():
        group = grouper(results, key)
        next(group)
        for value in values:
            group.send(value)
        group.send(None)

In [1]:
import collections

In [2]:
# 出租车队运营仿真
Event = collections.namedtuple('Event', 'time proc action')
def taxi_process(ident, trips, start_time = 0):
    """每次改变状态时创建事件，把控制权让给仿真器"""
    time = yield Event(start_time, ident, 'leave garage')
    for i in range(trips):
        time = yield Event(time, ident, 'pick up passenger')
        time = yield Event(time, ident, 'drop off passenger')
    yield Event(time, ident, 'going home')

In [3]:
# 驱动taxi_process协程
taxi = taxi_process(ident=13, trips=2, start_time=0)

In [4]:
print (next(taxi))

Event(time=0, proc=13, action='leave garage')


In [6]:
taxi.send(7)

Event(time=7, proc=13, action='pick up passenger')

In [7]:
taxi.send(30)

Event(time=30, proc=13, action='drop off passenger')

In [8]:
taxi.send(35)

Event(time=35, proc=13, action='pick up passenger')

In [9]:
taxi.send(83)

Event(time=83, proc=13, action='drop off passenger')

In [10]:
taxi.send(84)

Event(time=84, proc=13, action='going home')

In [11]:
taxi.send(1)

StopIteration: 

In [12]:
import queue

In [ ]:
class Simulator:
    def __init__(self, procs_map):
        self.events = queue.PriorityQueue()
        self.procs = dict(procs_map)

    def run(self, end_time):
        """排定并显示事件，直到时间结束"""
        # 排定各辆出租车的第一个事件
        for _, proc in sorted(self.procs.items()):
            first_event = next(proc)
            self.events.put(first_event)

        sim_time = 0
        while sim_time < end_time:
            if self.events.empty():
                print ('*** end of events ***')
                break
                
            current_event = self.events.get()
            sim_time, proc_id, previous_action = current_event
            print ('taxi:', proc_id, proc_id * '    ', current_event)
            active_proc = self.procs[proc_id]
            next_time = sim_time + compute_duration(previous_action)
            try:
                next_event = active_proc.send(next_time)
            except StopIteration:
                del self.procs[proc_id]
            else:
                self.events.put(next_event)
        else:
            msg = '*** end of simulation time: {} events pending ***'
            print (msg.format(self.events.qsize()))